In [2]:
import numpy as np
import pandas as pd
import datetime
import math

In [114]:
price = pd.read_csv('dam_out.csv')
price['date']=pd.to_datetime(price['date'],format="%Y-%m-%d")#"%d/%m/%Y")
price=price.drop(['date.1'],axis=1)
price=price.set_index([pd.DatetimeIndex(price['date']),'hour'])
price

date  market  price  volume  year  month  day  \
date       hour                                                       
2016-01-01 0    2016-01-01       1  40.10   477.5  2016      1    1   
           1    2016-01-01       1  40.08   486.6  2016      1    1   
           2    2016-01-01       1  40.06   495.6  2016      1    1   
           3    2016-01-01       1  40.08   496.6  2016      1    1   
           4    2016-01-01       1  40.07   502.9  2016      1    1   
...                    ...     ...    ...     ...   ...    ...  ...   
2021-01-29 13   2021-01-29       1  61.97   134.0  2021      1   29   
           23   2021-01-29       1  58.05    59.0  2021      1   29   
2021-01-30 17   2021-01-30       1  42.08    64.0  2021      1   30   
2021-01-31 3    2021-01-31       1  42.19   149.0  2021      1   31   
           21   2021-01-31       1  42.18    92.0  2021      1   31   

                 day_of_week  month_no    hour_ref  
date       hour                                     
2016-01-01 0               4   2016.01   weekday_0  
           1               4   2016.01   weekday_1  
           2               4   2016.01   weekday_2  
           3               4   2016.01   weekday_3  
           4               4   2016.01   weekday_4  
...                      ...       ...         ...  
2021-01-29 13              4   2021.01  weekday_13  
           23              4   2021.01  weekday_23  
2021-01-30 17              5   2021.01  weekend_17  
2021-01-31 3               6   2021.01   weekend_3  
           21              6   2021.01  weekend_21  

[44592 rows x 10 columns]

In [130]:
flow = pd.read_csv('flow.csv')

flow['date']=pd.to_datetime(flow['flowdate'],format="%Y-%m-%d")#"%d/%m/%Y")
flow=flow.drop(['flowdate','flowid','market'],axis=1)

flow=flow.set_index([pd.DatetimeIndex(flow['date'])])
flow['year']=flow.index.year
flow['month']=flow.index.month
flow['day']=flow.index.day
flow['day_of_week']=flow.index.dayofweek
flow['month_no']=flow['year']+flow['month']/100

flow['hour_ref'] = flow.apply(lambda x: ('weekend_' + str(x['hour']) if x['day_of_week']>4 else 'weekday_' + str(x['hour'])),axis=1)
flow=flow.set_index([pd.DatetimeIndex(flow['date']),'hour'])
flow['price']=price['price']
flow['value']=flow['price']*flow['flow']
flow=flow.drop(['date'],axis=1)
flow=flow.reset_index()
flow

,date,hour,flow,from,to,year,month,day,day_of_week,month_no,hour_ref,price,value
0,2016-01-02,13,50,BOT,ZIMA,2016,1,2,5,2016.01,weekend_13,50.05,2502.50
1,2016-01-02,14,63,ZIMA,ZIM,2016,1,2,5,2016.01,weekend_14,50.31,3169.53
2,2016-01-02,15,263,ZIM,ZAMZ,2016,1,2,5,2016.01,weekend_15,50.49,13278.87
3,2016-01-02,16,100,MOZN_EDM,ZIMA,2016,1,2,5,2016.01,weekend_16,56.07,5607.00
4,2016-01-02,17,120,ZIM,ZAMZ,2016,1,2,5,2016.01,weekend_17,56.06,6727.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
156456,2018-09-10,8,45,ZIM,ZAMZ,2018,9,10,0,2018.09,weekday_8,85.90,3865.50
156457,2018-09-10,9,190,MOZS,RSAN,2018,9,10,0,2018.09,weekday_9,85.62,16267.80
156458,2018-09-10,10,68,RSAS,NAM,2018,9,10,0,2018.09,weekday_10,56.85,3865.80
156459,2018-09-10,9,200,RSAS,NAM,2018,9,10,0,2018.09,weekday_9,85.62,17124.00


In [131]:
flowa=flow.copy().rename(columns={'to':'from','from':'to'})
flow.loc[
    (
        (flow['from']=='RSAN') & (flow['to']=='MOZS') |
        (flow['from']=='MOZS') & (flow['to']=='RSAN') |
        (flow['from']=='MOZN_EDM') & (flow['to']=='ZIMA') |
        (flow['from']=='ZIMA') & (flow['to']=='MOZN_EDM') |
        (flow['from']=='MOZN_HCB') & (flow['to']=='MOZN_EDM') |
        (flow['from']=='MOZN_EDM') & (flow['to']=='MOZN_HCB')
    ) 
    ,['to','from']]=flowa.loc[
    (
        (flow['from']=='RSAN') & (flow['to']=='MOZS') |
        (flow['from']=='MOZS') & (flow['to']=='RSAN') |
        (flow['from']=='MOZN_EDM') & (flow['to']=='ZIMA') |
        (flow['from']=='ZIMA') & (flow['to']=='MOZN_EDM') |
        (flow['from']=='MOZN_HCB') & (flow['to']=='MOZN_EDM') |
        (flow['from']=='MOZN_EDM') & (flow['to']=='MOZN_HCB')
    )     
    ,['to','from']]

In [132]:
f_to=flow[['date','hour','flow','value','to','year','month','day','day_of_week','month_no','hour_ref']].rename(columns={'to': 'node'})
f_from=flow[['date','hour','flow','value','from','year','month','day','day_of_week','month_no','hour_ref']].rename(columns={'from': 'node'})
f_from['flow']=-f_from['flow']

flows=pd.concat([f_to, f_from])

flows

,date,hour,flow,value,node,year,month,day,day_of_week,month_no,hour_ref
0,2016-01-02,13,50,2502.50,ZIMA,2016,1,2,5,2016.01,weekend_13
1,2016-01-02,14,63,3169.53,ZIM,2016,1,2,5,2016.01,weekend_14
2,2016-01-02,15,263,13278.87,ZAMZ,2016,1,2,5,2016.01,weekend_15
3,2016-01-02,16,100,5607.00,MOZN_EDM,2016,1,2,5,2016.01,weekend_16
4,2016-01-02,17,120,6727.20,ZAMZ,2016,1,2,5,2016.01,weekend_17
...,...,...,...,...,...,...,...,...,...,...,...
156456,2018-09-10,8,-45,3865.50,ZIM,2018,9,10,0,2018.09,weekday_8
156457,2018-09-10,9,-190,16267.80,RSAN,2018,9,10,0,2018.09,weekday_9
156458,2018-09-10,10,-68,3865.80,RSAS,2018,9,10,0,2018.09,weekday_10
156459,2018-09-10,9,-200,17124.00,RSAS,2018,9,10,0,2018.09,weekday_9


In [133]:
flows.loc[flows['node']=='ZIMA','node']='ZIM'
flows.loc[flows['node']=='MOZN_EDM','node']='MOZ'
flows.loc[flows['node']=='MOZN_HCB','node']='MOZ'
flows.loc[flows['node']=='MOZS','node']='MOZ'
flows.loc[flows['node']=='RSAS','node']='RSA'
flows.loc[flows['node']=='RSAN','node']='RSA'

In [134]:
flows=flows.reset_index().groupby(['date','hour','node','year','month','day','day_of_week','month_no','hour_ref']).sum().reset_index()
flows=flows.loc[flows['flow']!=0]
flows

,date,hour,node,year,month,day,day_of_week,month_no,hour_ref,index,flow,value
0,2016-01-01,0,ZAMC,2016,1,1,4,2016.01,weekday_0,5607,160,6416.00
1,2016-01-01,0,ZAMZ,2016,1,1,4,2016.01,weekday_0,11320,10,13233.00
2,2016-01-01,0,ZIM,2016,1,1,4,2016.01,weekday_0,5713,-170,6817.00
3,2016-01-01,1,ZAMC,2016,1,1,4,2016.01,weekday_1,5714,160,6412.80
4,2016-01-01,1,ZAMZ,2016,1,1,4,2016.01,weekday_1,11322,10,13226.40
...,...,...,...,...,...,...,...,...,...,...,...,...
137806,2018-09-10,10,BOT,2018,9,10,0,2018.09,weekday_10,312599,185,15178.95
137807,2018-09-10,10,MOZ,2018,9,10,0,2018.09,weekday_10,156255,173,9835.05
137808,2018-09-10,10,NAM,2018,9,10,0,2018.09,weekday_10,156458,68,3865.80
137809,2018-09-10,10,RSA,2018,9,10,0,2018.09,weekday_10,781758,-467,34280.55


In [135]:
# flows.loc[(flows['date']=='2016-01-15') & (flows['hour']==0)]

In [136]:
flow_monthly=flows.groupby(['month_no','year','month','hour', 'hour_ref','node']).sum().reset_index(['hour','month','year'])
flow_monthly.rename(columns={'flow': 'flow_sum'}, inplace=True)
flow_monthly=flow_monthly.drop(['day_of_week','day'],axis=1)
flow_monthly

year  month  hour    index  flow_sum      value
month_no hour_ref   node                                                 
2016.01  weekday_0  BOT   2016      1     0    16865       -71    3460.97
                    MOZ   2016      1     0   143656      1319  196209.27
                    NAM   2016      1     0     4724        50    2409.80
                    RSA   2016      1     0     7797      -104    5100.52
                    ZAMC  2016      1     0    49637      1391   65631.17
...                        ...    ...   ...      ...       ...        ...
2018.09  weekend_23 NAM   2018      9    23   621736       763   20522.38
                    RSA   2018      9    23  3417705     -1367   85556.42
                    ZAMC  2018      9    23   621210       162    4338.79
                    ZAMZ  2018      9    23  1242903      -195    5246.55
                    ZIM   2018      9    23   618448       -60    5475.36

[10009 rows x 6 columns]

In [137]:
flow_monthly.to_csv('dam_flow_monthly.csv')